## Check if we're using GPU OR CPU

In [ ]:
import tensorflow as tf 

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

## Listing Devices including GPU's with Tensorflow

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

## To Check GPU in Tensorflow


In [ ]:
tf.test.is_gpu_available()

# 1) Loading Dataset

In [ ]:
#To Load Datasets
from tensorflow.keras.datasets import mnist 

#load the MNIST training and test dataset 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 2) Inspecting our Dataset 

In [ ]:
#Display the number of samples in x_train, x_test, y_train, y_test 
print('The shape or dimensions of x_train', str(x_train.shape))

#Print the number of samples in our data 
print('Number of samples in training data: ', str(len(x_train)))
print('Number of labels in training data: ', str(len(y_train)))
print('Number of samples in test data: ', str(len(x_test)))
print('Number of labels in test data: ', str(len(y_test)))

# 3) Visualizing our image dataset

In [ ]:
# Let's view the 50 first images of the MNIST training dataset
import matplotlib.pyplot as plt

# Create figure and change size
figure = plt.figure()
plt.figure(figsize=(16,10))

# Set how many images we wish to see
num_of_images = 50 

# iterate index from 1 to 51
for index in range(1, num_of_images + 1):
    plt.subplot(5, 10, index).set_title(f'{y_train[index]}')
    plt.axis('off')
    plt.imshow(x_train[index], cmap='gray_r')

# 4) Preprocessing 

In [ ]:
#Let's store the number of rows and columns 
img_rows = x_train[0].shape[0]
img_columns = x_train[0].shape[1]

#Getting our data in the right shape needed for keras 
#We need to add a 4th dimension to our data thereby changing our 
#Original image shape of (60000, 28, 28) to (60000, 28, 28, 1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_columns, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_columns, 1)

#store the shape of a single image 
input_shape = (img_rows, img_columns, 1)

#change our image type to float32 data type 
x_train = x_train.astype('float32') #uint8 originally 
x_test = x_test.astype('float32')

#Normalize our data by changing the range from (0:255) to (0:1)
x_train /= 255.0
x_test /= 255.0

print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print(img_rows, img_columns)

## One-Hot Encoding (y-labels)

In [ ]:
from tensorflow.keras.utils import to_categorical

#Now we one hot encode outputs 
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#Let's count thee number of columns in our hot encoded matrix 
print("Number of classes: ", str(y_test.shape[1]))

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

In [ ]:
#Look at our raw data
y_train[0]

# 5) Building Convolutional Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD 

model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu', strides = (2,2)))
model.add(MaxPooling2D(pool_size = (2, 2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = SGD(0.001), metrics=['accuracy'])

print(model.summary())

# 6) Training The Model

In [ ]:
batch_size = 128
epochs = 25

import time

start = time.time()

history = model.fit(x_train,
                    y_train, 
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test))

score = model.evaluate(x_test, y_test, verbose = 0)
print('Test Loss: ', score[0])
print('Test Accuracy: ', score[1])
print("Total time: ", time.time() - start, "seconds") 

# 7) Ploting Loss and Accuracy

In [ ]:
# Plotting our loss charts
import matplotlib.pyplot as plt

# Use the History object we created to get our saved performance results
history_dict = history.history

# Extract the loss and validation losses
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

# Get the number of epochs and create an array up to that number using range()
epochs = range(1, len(loss_values) + 1)

# Plot line charts for both Validation and Training Loss
line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Plotting our accuracy charts
import matplotlib.pyplot as plt

history_dict = history.history

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_acc_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
import numpy as np
import seaborn as sns 

# Predict the values from the testing dataset
Y_pred = model.predict(x_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert testing observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1)
# compute the confusion matrix
confusion_mtx = tf.math.confusion_matrix(Y_true, Y_pred_classes) 

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='g')